In [1]:
from models.sdf_gat import *

In [2]:
qnet = SDFGATQNet(3, sdim=1, fdim=12, hdim=64).to(device)

In [3]:
nb = 3
bs = 7
ch = 1
h, w = 96, 96
f = 12

sdf_st = torch.rand([bs, nb, ch, h, w]).to(device)
sdf_g = torch.rand([bs, nb, ch, h, w]).to(device)
feature_st = torch.rand([bs, nb, f]).to(device)
feature_g = torch.rand([bs, nb, f]).to(device)
obs_st = [sdf_st, feature_st]
obs_g = [sdf_g, feature_g]

nb_st = torch.randint(0, nb+1, (bs, 1)).to(device)
nb_g = torch.randint(0, nb+1, (bs, 1)).to(device)

In [4]:
atn_in, atn_g = qnet(obs_st, obs_g, nb_st, nb_g)

In [5]:
nb_st

tensor([[1],
        [3],
        [0],
        [3],
        [3],
        [0],
        [1]], device='cuda:0')

In [6]:
nb_g

tensor([[1],
        [3],
        [0],
        [2],
        [0],
        [2],
        [2]], device='cuda:0')

In [10]:
atn_in[3]

tensor([[0.0000, 0.4931, 0.5069, 0.0000, 0.0000, 0.0000],
        [0.5196, 0.0000, 0.4804, 0.0000, 0.0000, 0.0000],
        [0.5264, 0.4736, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [13]:
atn_g[3]

tensor([[0.0000, 0.0000, 0.0000, 0.4086, 0.5914, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4086, 0.5914, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4815, 0.5185, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [12]:
torch.matmul(atn_g[0], torch.ones(2*nb, f).to(device)).shape

torch.Size([6, 12])

In [12]:
adj_inscene, adj_goal = qnet.generate_adj(nb_st, nb_g)

In [16]:
x = torch.zeros([3,2,10])

In [17]:
x.shape

torch.Size([3, 2, 10])

In [21]:
x.reshape(3, 1, *x.shape[-2:]).shape

torch.Size([3, 1, 2, 10])

In [ ]:
import torchvision.models as models

In [ ]:
resnet50 = models.resnet50(pretrained=True).cuda()

In [ ]:
f = resnet50(torch.tensor(state[0].transpose([2,0,1])).view([1, 3, 480, 480]).type(torch.cuda.FloatTensor))

In [ ]:
f.shape

In [ ]:
f.cpu().detach().numpy().shape

In [ ]:
(1, *f.shape)

In [ ]:
y.shape

In [ ]:
q = np.random.random([480,480])

In [ ]:
y[:, q>0.5] = 0

In [ ]:
plt.imshow(y.cpu().permute([1,2,0]))

In [ ]:
torch.cat([y, y]).shape

In [ ]:
y = copy.deepcopy(x[0])

In [ ]:
y[y<0.9]=0

In [ ]:
plt.imshow(y.cpu().permute([1,2,0]))

In [ ]:
import sys
sys.path.append('../../ur5_mujoco')

from ur5_env import *
from object_env import *

from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance_matrix

from sdf_module import *

In [ ]:
sdf = SDFModule() #rgb_feature=True, ucn_feature=False, resnet_feature=True)

In [ ]:
from PIL import Image

In [ ]:
im1 = Image.open('test_scenes/detection/432.png')
im2 = Image.open('test_scenes/detection/441.png')

In [ ]:
plt.imshow(np.array(im1)[:, :480])

In [ ]:
s1 = np.array(im1)[:, :480] / 255.
s2 = np.array(im2)[:, :480] / 255.

sdf1, sdf_raw1, feature1 = sdf.get_sdf_features(s1)
sdf2, sdf_raw2, feature2 = sdf.get_sdf_features(s2)

In [ ]:
for i in range(3):
    plt.imsave('test_scenes/sr1_%d.png'%i, np.clip(sdf_raw1[i], -0.1, 0.04), cmap='seismic', vmin=-0.15, vmax=0.07)
    plt.imsave('test_scenes/sr2_%d.png'%i, np.clip(sdf_raw2[i], -0.1, 0.04), cmap='seismic', vmin=-0.15, vmax=0.07)

In [ ]:
plt.imshow(sdf1[0])

In [ ]:
plt.imshow(np.abs(sdf1[0])<0.005)

In [ ]:
sdf1[0].max()

In [ ]:
plt.imshow(sdf1[0], cmap='seismic', vmin=-0.5, vmax=0.5)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.05, 0.04), cmap='seismic', vmin=-0.1, vmax=0.1)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.1, 0.04), cmap='seismic', vmin=-0.15, vmax=0.07)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.12, 0.04), cmap='seismic', vmin=-0.2, vmax=0.12)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.1, 0.04), cmap='seismic', vmin=-0.2, vmax=0.1)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.05, 0.04), cmap='coolwarm', vmin=-0.08, vmax=0.05)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.05, 0.04), cmap='spectral', vmin=-0.1, vmax=0.1)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.05, 0.04), cmap='jet', vmin=-0.1, vmax=0.1)

In [ ]:
plt.imshow(np.clip(sdf1[0], -0.05, 0.04), cmap='rainbow', vmin=-0.1, vmax=0.1)

In [ ]:
env = UR5Env(render=True, camera_height=480, camera_width=480, control_freq=5, data_format='NHWC', xml_ver=0)
env = objectwise_env(env, num_blocks=3, mov_dist=0.08, detection=True)

In [ ]:
state = env.reset()

In [ ]:
state[0].shape

In [ ]:
state[1].shape

In [ ]:
st = time.time()
s0, s0_raw, f0 = sdf.get_sdf_features(state[0], rotate=True)
sg, sg_raw, fg = sdf.get_sdf_features(state[1], rotate=True)
et = time.time()
print(et-st)

In [ ]:
s0[0].max()

In [ ]:
plt.imshow(np.abs(s0[0])<0.005)

In [ ]:
x = np.abs(s0[0])<0.01

In [ ]:
plt.imshow(x)

In [ ]:
from skimage.morphology import convex_hull_image

In [ ]:
plt.imshow(convex_hull_image(x))

In [ ]:
from scipy.spatial import ConvexHull

In [ ]:
py, px = np.where(x) 
p = np.array(list(zip(px,py)))

In [ ]:
hull = ConvexHull(p)

In [ ]:
hull.simplices

In [ ]:
hull.vertices

In [ ]:
plt.fill(p[hull.vertices, 0], p[hull.vertices,1], 'k', alpha=0.3)

In [ ]:
from scipy.ndimage import morphology

In [ ]:
from scipy.ndimage import morphology as morphology

In [ ]:
plt.imshow(morphology.binary_fill_holes(x))

In [ ]:
c, h = cv2.findContours(x, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

In [ ]:
px, py = np.where(x) 
p = np.array(list(zip(px,py)))

In [ ]:
hull = cv2.convexHull(p)

In [ ]:
dst = np.zeros_like(x)
plt.imshow(cv2.drawContours(dst, [hull], 0, [1], 2))

In [ ]:
#dst = np.zeros_like(x)
dst = np.zeros([96,96,3])
for i in c:
    hull = cv2.convexHull(i, clockwise=True)
    cv2.drawContours(dst, [hull], 0, (0, 0, 255), 2)

In [ ]:
hull

In [ ]:
plt.imshow(dst)

In [ ]:
plt.imshow(((np.abs(s0[0])<0.01) * np.ones([3,1,1])).transpose([1,2,0]))

In [ ]:
np.concatenate(f0, 1).shape

In [ ]:
np.concatenate(f0, 1).shape

In [ ]:
np.concatenate(f0[0], 1).shape

In [ ]:
import pylab
NUM_COLORS = 5

cm = pylab.get_cmap('gist_rainbow')
for i in [0,2,4,1,3]:
    color = cm(1.*i/NUM_COLORS)  # color will now be an RGBA tuple
    print(color)
    #plt.imshow(np.ones([6,6,3])*color[:3])
    #plt.show()

# or if you really want a generator:
cgen = (cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS))

In [ ]:
st = time.time()
s0, s0_raw, (r0, f0) = sdf.get_sdf_features(state[0], rotate=True)
sg, sg_raw, (rg, fg) = sdf.get_sdf_features(state[1], rotate=True)
et = time.time()
print(et-st)

In [ ]:
def norm(sdf):
    sdf_positive = sdf - sdf.min()
    return sdf_positive / sdf_positive.max()

In [ ]:
plt.imshow(norm(sg[0]))

In [ ]:
plt.imshow(norm(sg.transpose([1,2,0])))

In [ ]:
ss = sg.copy()
ss[sg<0] = 0
plt.imshow(norm(ss.transpose([1,2,0])))

In [ ]:
plt.imshow((sg.transpose([1,2,0])>0).astype(np.float))

In [ ]:
plt.imshow(norm(sg.transpose([1,2,0]))+(sg.transpose([1,2,0])>0).astype(np.float))

In [ ]:
plt.imshow(norm(sg.transpose([1,2,0])+2*(sg.transpose([1,2,0])>0).astype(np.float)))

In [ ]:
cm(0)[:3]

In [ ]:
newsdf = norm(sg+2*(sg>0).astype(np.float))

In [ ]:
newsdf = norm(np.concatenate([s0, sg])+4*(np.concatenate([s0, sg])>0).astype(np.float))
empty = np.zeros([96,96,3])
for i in range(6):#len(newsdf)):
    empty += np.expand_dims(newsdf[i], 2) * np.array(cm(i/5)[:3])
plt.imshow(norm(empty))

In [ ]:
plt.imshow(np.expand_dims(newsdf[0], 2) * np.array(cm(0)[:3]))

In [ ]:
plt.imshow(np.clip(s0_raw[0], -100, 100))

In [ ]:
plt.imshow(s0_raw[0])

In [ ]:
plt.imshow(s0_raw.transpose([1,2,0]))

In [ ]:
linear_sum_assignment(distance_matrix(rg, r0)) 

In [ ]:
linear_sum_assignment(distance_matrix(fg, f0)) 

In [ ]:
st = time.time()
s0, s0_raw, (r0, f0) = sdf.get_sdf_features(state[0])
sg, sg_raw, (rg, fg) = sdf.get_sdf_features(state[1])
et = time.time()
print(et-st)

In [ ]:
linear_sum_assignment(distance_matrix(rg, r0)) 

In [ ]:
linear_sum_assignment(distance_matrix(fg, f0)) 

In [ ]:
mt, ft = sdf.get_masks(state[0], data_format='HWC', rotate=True)
mg, fg = sdf.get_masks(state[1], data_format='HWC', rotate=True)


In [ ]:
plt.figure(figsize=(8, 30))
N = 10
ax1 = plt.subplot(N, 2, 1)
ax2 = plt.subplot(N, 2, 2)
ax1.imshow(state[0])
ax2.imshow(state[1])
for i in range(1, N):
    _ax1 = plt.subplot(N, 2, 2*i+1)
    _ax2 = plt.subplot(N, 2, 2*i+2)
    _ax1.imshow(ft[i-1])
    _ax2.imshow(fg[i-1])

In [ ]:
sdf.object_matching((r0, f0), (rg, fg))

In [ ]:
s_align = s0[sdf.object_matching((r0, f0), (rg, fg))]

In [ ]:
np.round

In [ ]:
px, py = np.where(s_align[0]>0)
px = px.mean().round()
py = py.mean().round()
print(px, py)

In [ ]:
gx, gy = np.where(sg[0]>0)
gx = gx.mean().round()
gy = gy.mean().round()
print(gx, gy)

In [ ]:
from PIL import Image, ImageDraw

In [ ]:
state[0].shape

In [ ]:
plt.imshow(np.concatenate([state[0], state[1]], 1))

In [ ]:
im = Image.fromarray((np.concatenate([state[0], state[1]], 1)*255).astype(np.uint8))
draw = ImageDraw.Draw(im)
for i in range(3):
    px, py = np.where(s_align[i]>0)
    px = px.mean().round()
    py = py.mean().round()
    gx, gy = np.where(sg[i]>0)
    gx = gx.mean().round()
    gy = gy.mean().round()
    draw.line((5*py, 5*px, 5*gy+480, 5*gx), fill=128, width=3)
im.show()

In [ ]:
plt.imshow(s_align[0]>0)

In [ ]:
st = time.time()
_ = sdf.get_masks(state[0])
_ = sdf.get_masks(state[1])
et = time.time()
print(et-st)

In [ ]:
st = time.time()
image = state[0]
# image = image.transpose([2, 0, 1])

im_tensor = torch.from_numpy(image).type(torch.float32).permute(2, 0, 1).unsqueeze(0).cuda()
features = sdf.network(im_tensor, None).detach()
out_label, selected_pixels = clustering_features(features, num_seeds=100)

et = time.time()
print(et-st)

In [ ]:
features.shape

In [ ]:
torch.cat((features, features)).shape

In [ ]:
out_labem, sp = clustering_features(torch.cat((features, features)), num_seeds=100)

In [ ]:
out_labem.shape

In [ ]:
out_labem

In [ ]:
plt.imshow(out_labem[0])

In [ ]:
plt.imshow(out_labem[1])

In [ ]:
im_tensor.shape

In [ ]:
plt.imshow(state[0])

In [ ]:
plt.imshow(s0.transpose([1,2,0]))

In [ ]:
plt.imshow(state[1])

In [ ]:
plt.imshow(sg[:3].transpose([1,2,0]))

In [ ]:
np.array(f0).dot(np.array(fg).T)

In [ ]:
distance_matrix(f0, fg)

In [ ]:
xn = f0 / np.linalg.norm(f0, axis=1).reshape(3, 1)
yn = fg / np.linalg.norm(fg, axis=1).reshape(3, 1)

In [ ]:
xn.dot(yn.T)

In [ ]:
xn.dot(yn.T).argmax(1)

In [ ]:
distance_matrix(xn, yn)

In [ ]:
row_ind, col_ind = linear_sum_assignment(distance_matrix(xn, yn))
col_ind

In [ ]:
r0n = r0 / np.linalg.norm(r0, axis=1).reshape(3, 1)
rgn = rg / np.linalg.norm(rg, axis=1).reshape(3, 1)

In [ ]:
r0n.dot(rgn.T)

In [ ]:
r0n.dot(rgn.T).argmax(1)

In [ ]:
distance_matrix(r0, rg)

In [ ]:
distance_matrix(r0n, rgn)

In [ ]:
row_ind, col_ind = linear_sum_assignment(distance_matrix(r0, rg))
print(col_ind)

In [ ]:
new0 = np.concatenate([r0, f0], axis=1)
newg = np.concatenate([rg, fg], axis=1)

In [ ]:
new0n = new0 / np.linalg.norm(new0, axis=1).reshape(3, 1)
newgn = newg / np.linalg.norm(newg, axis=1).reshape(3, 1)

In [ ]:
new0n.dot(newgn.T)

In [ ]:
new0n.dot(newgn.T).argmax(1)

In [ ]:
idx = new0n.dot(newgn.T).argmax(1)
new0n.dot(newgn[idx].T).argmax(1)

In [ ]:
ss, sg = sdf.get_aligned_sdfs(state[0], state[1])

In [ ]:
plt.imshow(ss.transpose([1,2,0]))

In [ ]:
plt.imshow(sg.transpose([1,2,0]))

In [ ]:
import cv2

In [ ]:
plt.imshow(state[0][:,:,0])

In [ ]:
x = cv2.resize(state[0][:,:,0], (96, 96))
plt.imshow(x)

In [ ]:
x.shape

In [ ]:
x = cv2.resize(s0[0], (96, 96))
plt.imshow(x)

In [ ]:
s0.shape

In [ ]:
plt.imshow(s0[1]>0)

In [ ]:
[px], [py] = np.where(s0[1]==s0[1].max())

In [ ]:
px, py

In [ ]:
z = np.zeros([96, 96])
z[px, py] = 1
plt.imshow(z)

In [ ]:
import numpy as np

In [ ]:
x = np.random.random([4, 5])
y = np.random.random([3, 5])

In [ ]:
x

In [ ]:
y

In [ ]:
x.dot(y.T)

In [ ]:
x.dot(y.T).argmax(1)

In [ ]:
x.dot(y.T).argmax(0)

In [ ]:
x[x.dot(y.T).argmax(0)]

In [ ]:
xn = x / np.linalg.norm(x, axis=1).reshape(4, 1)
yn = y / np.linalg.norm(y, axis=1).reshape(3, 1)

In [ ]:
xn.dot(yn.T)

In [ ]:
xn.dot(yn.T).argmax(1)